In [1]:
import pandas as pd 
import numpy as np
import os
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import altair as alt
pd.set_option('max_columns', None)

C:\Users\Zach\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


Cumulative Average code  - Opponent Stats

In [2]:
def convert_forwards_bigs(x):
    if x=='C' or x=='F' or x=='SF' or x=='PF':
        return 'Big_man'
    elif x=='G' or x=='PG'or x=='SG':
        return 'Guard'
    else:
        ### Some missing positions (ATH - Athlete OR NAN)
        return x




In [14]:

### Shows all the columns in a dataframe
pd.set_option('max_columns', None)
#### Disables chained assingments without the need to create a copy
pd.options.mode.chained_assignment = None


#### Individual games points to starters and reserves 
opponents_points_to_starters_and_reserves= pd.DataFrame()
opponents_points_to_starters_and_reserves_position = pd.DataFrame()
### Use to append the opponent stats allowed to starters and reserves
Rolling_average_grouped_starters_reserves = pd.DataFrame()
### Stores total points by opponent for each game
total_opponent_points_allowed = pd.DataFrame()
#### players ppg 
player_box_with_points_avergae= pd.DataFrame()
### team-players stats
all_teams_box_score_df = pd.DataFrame()

merged_team_opp_stats = pd.DataFrame()
merged_team_opp_stats_with_pos = pd.DataFrame()


# for file in tqdm(file_names[:4]):### Limit number of seasons 
#   if len((pd.read_csv('{}/{}'.format(teams_path,file))))>0:
#     df= pd.read_csv('{}/{}'.format(teams_path,file),index_col=0)
    # display(df.head())
    # print(df.columns)

for year in np.arange(2019,2023):
    df = pd.read_csv("src\\team_player_boxscore_merged\\{}.csv".format(year),index_col = 0)
    players_list = list(df.athlete_display_name.unique())

    # ############################################ Team statistics #########################################################################

    # ### gets the cumulative average points and shifts it to the next game to use for predicting (does not include game in the current row.)  
    df['points_per_min']=df['pts']/df['min']
    df=df.assign(prev_player_ppg=df.groupby('athlete_display_name', sort=False)['pts'].transform(lambda x: x.expanding().mean().round(2).shift(1)))
    df=df.assign(prev_player_points_per_min=df.groupby('athlete_display_name', sort=False)['points_per_min'].transform(lambda x: x.expanding().mean().round(3).shift(1)))
    player_box_with_points_avergae=player_box_with_points_avergae.append(df)

    list_of_teams = list(df.team_abbreviation.unique())
    for team in list_of_teams: 

        team_df = df[df.team_abbreviation==team]
        date_total_points = team_df.groupby('game_date')['pts'].sum().reset_index().rename(columns = {'pts':'game_total_points'})
        team_df = pd.merge(team_df,date_total_points,on='game_date')

        ### Gets the game number for each team.
        games_list = list(team_df[team_df.team_abbreviation==team]["game_id"].unique())
        game_num = list(np.arange(1,len(games_list)+1))
        game_number_df = (pd.DataFrame((list(zip(games_list,game_num))),columns = ['game_id','game_number']))
        game_number_df["team_abbreviation"]=team

        # game_num_dates_all_teams=game_num_dates_all_teams.append(game_number_df)
        team_df = pd.merge(team_df,game_number_df,on=['team_abbreviation','game_id'])
        team_df["Cleaned_position_bigs_guards"] = team_df['athlete_position_abbreviation'].apply(convert_forwards_bigs)

        ### Append to dataframe with all teams 
        ## Add cleaned_position_bigs_guards to all_teams_box_score_df
        all_teams_box_score_df = all_teams_box_score_df.append(team_df)
    
    
      ######################################### Stats against opponents ######################################################################################

      #### Get a rolling average for points allowed by opponents for each team split into starters and reserves 
      ### Create a function for the desired rolling average window. 
        rolling_average_widow = 3
        opponent_df = df[df.opponent_abbrev==team].sort_values(by = "game_date")

        opponent_df["Cleaned_position_bigs_guards"] = opponent_df['athlete_position_abbreviation'].apply(convert_forwards_bigs)


        ##### For starters/reserves and position

        grouped_position_opponent_df = opponent_df.loc[:,['season','min','largest_lead','pts']+["game_date","game_id",'opponent_abbrev','starter','Cleaned_position_bigs_guards']].groupby(
          ["game_date","game_id",'opponent_abbrev','starter','Cleaned_position_bigs_guards']).agg(
          {'season':'mean','min':'mean','pts':'sum'}
        )

        grouped_position_opponent_df = grouped_position_opponent_df.rename(columns={"pts":"pts_allowed"})
        grouped_position_opponent_df['season'] = grouped_position_opponent_df.season.astype(int)
        opponents_points_to_starters_and_reserves_position = opponents_points_to_starters_and_reserves_position.append(grouped_position_opponent_df)
        grouped_position_opponent_df = grouped_position_opponent_df.reset_index()
        #### Starting bigs
        grouped_bigs_starters =  grouped_position_opponent_df[(grouped_position_opponent_df['Cleaned_position_bigs_guards']=='Big_man') & (grouped_position_opponent_df['starter']==True)]
        grouped_bigs_starters["cum_average_opp_points_to_role_position"]=(grouped_bigs_starters['pts_allowed'].cumsum(axis=0) / (grouped_bigs_starters['pts_allowed'] != 0).cumsum(axis=0)).round(3).shift(1) 
        grouped_bigs_starters["cum_average_opp_min_allowed_to_role_position"]=(grouped_bigs_starters['min'].cumsum(axis=0) / (grouped_bigs_starters['min'] != 0).cumsum(axis=0)).round(3).shift(1)
        cumulative_averages_grouped_bigs_starters = grouped_bigs_starters.loc[:, ~grouped_bigs_starters.columns.isin(["min"	,"pts_allowed"])]


        #### Starting guards
        grouped_guards_starters =  grouped_position_opponent_df[(grouped_position_opponent_df['Cleaned_position_bigs_guards']=='Guard') & (grouped_position_opponent_df['starter']==True)]
        grouped_guards_starters["cum_average_opp_points_to_role_position"]=(grouped_guards_starters['pts_allowed'].cumsum(axis=0) / (grouped_guards_starters['pts_allowed'] != 0).cumsum(axis=0)).round(3).shift(1) 
        grouped_guards_starters["cum_average_opp_min_allowed_to_role_position"]=(grouped_guards_starters['min'].cumsum(axis=0) / (grouped_guards_starters['min'] != 0).cumsum(axis=0)).round(3).shift(1)
        cumulative_averages_grouped_guards_starters = grouped_guards_starters.loc[:, ~grouped_guards_starters.columns.isin(["min"	,"pts_allowed"])]

        #### Reserve bigs 
        grouped_bigs_reserves =  grouped_position_opponent_df[(grouped_position_opponent_df['Cleaned_position_bigs_guards']=='Big_man') & (grouped_position_opponent_df['starter']==False)]
        grouped_bigs_reserves["cum_average_opp_points_to_role_position"]=(grouped_bigs_reserves['pts_allowed'].cumsum(axis=0) / (grouped_bigs_reserves['pts_allowed'] != 0).cumsum(axis=0)).round(3).shift(1) 
        grouped_bigs_reserves["cum_average_opp_min_allowed_to_role_position"]=(grouped_bigs_reserves['min'].cumsum(axis=0) / (grouped_bigs_reserves['min'] != 0).cumsum(axis=0)).round(3).shift(1)
        cumulative_averages_grouped_bigs_reserves = grouped_bigs_reserves.loc[:, ~grouped_bigs_reserves.columns.isin(["min"	,"pts_allowed"])]


        #### Reserve guards
        grouped_guards_reserves =  grouped_position_opponent_df[(grouped_position_opponent_df['Cleaned_position_bigs_guards']=='Guard') & (grouped_position_opponent_df['starter']==False)]
        grouped_guards_reserves["cum_average_opp_points_to_role_position"]=(grouped_guards_reserves['pts_allowed'].cumsum(axis=0) / (grouped_guards_reserves['pts_allowed'] != 0).cumsum(axis=0)).round(3).shift(1) 
        grouped_guards_reserves["cum_average_opp_min_allowed_to_role_position"]=(grouped_guards_reserves['min'].cumsum(axis=0) / (grouped_guards_reserves['min'] != 0).cumsum(axis=0)).round(3).shift(1)
        cumulative_averages_grouped_guards_reserves = grouped_guards_reserves.loc[:, ~grouped_guards_reserves.columns.isin(["min"	,"pts_allowed"])]

        starters_reserves__position_cumulative_averages= cumulative_averages_grouped_bigs_starters.append([cumulative_averages_grouped_guards_starters,
                                                                                                         cumulative_averages_grouped_bigs_reserves,
                                                                                                         cumulative_averages_grouped_guards_reserves
                                                                                                         ]).sort_values(by='game_date')



        #### For only starters and reserves 
        grouped_opponent_df = opponent_df.loc[:,['season','min','largest_lead','pts']+["game_date","game_id",'opponent_abbrev','starter']].groupby(
          ["game_date","game_id",'opponent_abbrev','starter']).agg(
          {'season':'mean','min':'mean','largest_lead':'mean','pts':'sum'}
        )

        grouped_opponent_df = grouped_opponent_df.rename(columns={"largest_lead":"largest_lead_entire_game","pts":"pts_allowed"})
        ### individual games - points to starters and reserves in grouped_opponent_df
        grouped_opponent_df['season'] = grouped_opponent_df.season.astype(int)
        ### Append to big dataframe with all opponents_points allowed
        opponents_points_to_starters_and_reserves = opponents_points_to_starters_and_reserves.append(grouped_opponent_df)

        #### Get total points for each opponent. 
        grouped_opponent_df_all_roles = opponent_df.loc[:,['season','pts']+["game_date","game_id",'opponent_abbrev']].groupby(
          ["game_date","game_id",'opponent_abbrev']).agg(
          {'season':'mean','pts':'sum'}
        )
        grouped_opponent_df_all_roles = grouped_opponent_df_all_roles.rename(columns={"pts":"total_pts_allowed"})
        grouped_opponent_df_all_roles['season'] = grouped_opponent_df_all_roles.season.astype(int)
        ### Looks at prev games average
        grouped_opponent_df_all_roles['prev_total_opp_ppg'] = (grouped_opponent_df_all_roles['total_pts_allowed'].cumsum(axis=0) / (grouped_opponent_df_all_roles['total_pts_allowed'] != 0).cumsum(axis=0)).round(3).shift(1) 

        total_opponent_points_allowed = total_opponent_points_allowed.append(grouped_opponent_df_all_roles)

       ###################################################### starters ################################################################
        grouped_starters = grouped_opponent_df.reset_index()[grouped_opponent_df.reset_index().starter == True]
        ### Cumulative Averages - Starters
        grouped_starters["cum_average_opp_points_to_role"]=(grouped_starters['pts_allowed'].cumsum(axis=0) / (grouped_starters['pts_allowed'] != 0).cumsum(axis=0)).round(3).shift(1) 
        grouped_starters["cum_average_opp_min_allowed_to_role"]=(grouped_starters['min'].cumsum(axis=0) / (grouped_starters['min'] != 0).cumsum(axis=0)).round(3).shift(1)


        ### Rolling Averages - starters
        grouped_starters["prev_{}_games_opp_min_allowed_to_role".format(rolling_average_widow)]=grouped_starters['min'].rolling(rolling_average_widow).mean().round(3).shift(1)
        grouped_starters["prev_{}_games__opp_points_allowed_to_role".format(rolling_average_widow)]=grouped_starters['pts_allowed'].rolling(rolling_average_widow).mean().round(3).shift(1)

        ### does not include totals, only rolling averages for starters
        rolling_averages_grouped_starters = grouped_starters.loc[:, ~grouped_starters.columns.isin(["min"	,"largest_lead_entire_game"	,"pts_allowed"])]

        ############################################################################ Reserves ###########################################################
        grouped_reserves = grouped_opponent_df.reset_index()[grouped_opponent_df.reset_index().starter == False]
        ### Cumulative Averages - reserves
        grouped_reserves["cum_average_opp_points_to_role"]=(grouped_reserves['pts_allowed'].cumsum(axis=0) / (grouped_reserves['pts_allowed'] != 0).cumsum(axis=0)).round(3).shift(1)
        grouped_reserves["cum_average_opp_min_allowed_to_role"]=(grouped_reserves['min'].cumsum(axis=0) / (grouped_reserves['min'] != 0).cumsum(axis=0)).round(3).shift(1)

        ### Rolling Averages - reserves
        grouped_reserves["prev_{}_games_opp_min_allowed_to_role".format(rolling_average_widow)]=grouped_reserves['min'].rolling(rolling_average_widow).mean().round(3).shift(1)
        grouped_reserves["prev_{}_games__opp_points_allowed_to_role".format(rolling_average_widow)]=grouped_reserves['pts_allowed'].rolling(rolling_average_widow).mean().round(3).shift(1)

        ### Gets rid of the columns we took the averages from 
        rolling_averages_grouped_reserves = grouped_reserves.loc[:, ~grouped_reserves.columns.isin(["min"	,"largest_lead_entire_game"	,"pts_allowed"])]


        starters_reserves_rolling_averages = rolling_averages_grouped_starters.append(rolling_averages_grouped_reserves).sort_values(by='game_date')

        ### Merge total opponent points with the rolling and cumulative averages
        starters_reserves_rolling_averages = pd.merge(starters_reserves_rolling_averages,total_opponent_points_allowed,on =['game_date','game_id','opponent_abbrev','season'])

        ### Append to larger df for all teams starters and reserves rolling average 
        Rolling_average_grouped_starters_reserves = Rolling_average_grouped_starters_reserves.append(starters_reserves_rolling_averages)


        #### Merge cumulative average with starters_reserves_rolling_averages
        starters_reserves_pos_rolling_averages = pd.merge(starters_reserves__position_cumulative_averages,Rolling_average_grouped_starters_reserves,on =['game_date','game_id','opponent_abbrev','season','starter'])

    ### Uncomment first one if you do not want to include position (which is missing some values in many teams and seasons) - and limit the files
    merged_team_opp_stats = merged_team_opp_stats.append(pd.merge(all_teams_box_score_df,Rolling_average_grouped_starters_reserves, on =['game_date','game_id','opponent_abbrev','starter','season']))
#     merged_team_opp_stats_with_pos = merged_team_opp_stats_with_pos.append(pd.merge(all_teams_box_score_df,starters_reserves_pos_rolling_averages, on =['game_date','game_id','opponent_abbrev','starter','season','Cleaned_position_bigs_guards']))



merged_team_opp_stats




,team_short_display_name,game_id,starting_lineup,athlete_display_name,min,oreb,dreb,reb,ast,stl,blk,to,pf,pts,starter,ejected,did_not_play,active,athlete_jersey,athlete_id,athlete_short_name,athlete_headshot_href,athlete_position_name,athlete_position_abbreviation,team_name,team_logo,team_id,team_abbreviation,team_color,season,season_type,game_date,FGM,FGA,3PM,3PA,FTM,FTA,largest_lead,home_away,opponent_id,opponent_abbrev,points_per_min,prev_player_ppg,prev_player_points_per_min,game_total_points,game_number,Cleaned_position_bigs_guards,cum_average_opp_points_to_role,cum_average_opp_min_allowed_to_role,prev_3_games_opp_min_allowed_to_role,prev_3_games__opp_points_allowed_to_role,total_pts_allowed,prev_total_opp_ppg
0,Auburn,401082996,"{'Bryce Brown', 'Horace Spencer', 'Malik Dunba...",Chuma Okeke,36,1,4,5,1,4,1,2,2,22,True,False,False,False,5.0,4278052,C. Okeke,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,Tigers,https://a.espncdn.com/i/teamlogos/ncaa/500/2.png,2,AUB,03244d,2019,2,2019-03-09,9,14,3,5,1,1,6.0,AWAY,2633,TENN,0.611111,11.50,0.370,84,5,Big_man,63.5,32.650,32.733,61.333,84,74.5
1,Auburn,401082996,"{'Bryce Brown', 'Horace Spencer', 'Malik Dunba...",Horace Spencer,19,3,2,5,0,0,2,0,3,2,True,False,False,False,0.0,3908520,H. Spencer,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,Tigers,https://a.espncdn.com/i/teamlogos/ncaa/500/2.png,2,AUB,03244d,2019,2,2019-03-09,0,1,0,0,2,2,6.0,AWAY,2633,TENN,0.105263,3.00,0.276,84,5,Big_man,63.5,32.650,32.733,61.333,84,74.5
2,Auburn,401082996,"{'Bryce Brown', 'Horace Spencer', 'Malik Dunba...",Malik Dunbar,20,0,3,3,2,2,0,0,0,5,True,False,False,False,4.0,4278055,M. Dunbar,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,Tigers,https://a.espncdn.com/i/teamlogos/ncaa/500/2.png,2,AUB,03244d,2019,2,2019-03-09,1,3,1,2,2,4,6.0,AWAY,2633,TENN,0.250000,9.00,0.405,84,5,Guard,63.5,32.650,32.733,61.333,84,74.5
3,Auburn,401082996,"{'Bryce Brown', 'Horace Spencer', 'Malik Dunba...",Jared Harper,31,1,2,3,8,1,0,2,3,16,True,False,False,False,1.0,4066248,J. Harper,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,Tigers,https://a.espncdn.com/i/teamlogos/ncaa/500/2.png,2,AUB,03244d,2019,2,2019-03-09,5,13,2,7,4,4,6.0,AWAY,2633,TENN,0.516129,15.50,0.445,84,5,Guard,63.5,32.650,32.733,61.333,84,74.5
4,Auburn,401082996,"{'Bryce Brown', 'Horace Spencer', 'Malik Dunba...",Bryce Brown,33,0,3,3,2,2,0,1,1,16,True,False,False,False,2.0,3908521,B. Brown,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,Tigers,https://a.espncdn.com/i/teamlogos/ncaa/500/2.png,2,AUB,03244d,2019,2,2019-03-09,4,10,4,9,4,5,6.0,AWAY,2633,TENN,0.484848,11.75,0.367,84,5,Guard,63.5,32.650,32.733,61.333,84,74.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2141,Wisconsin,401364469,"{'Johnny Davis', 'Tyler Wahl', 'Chucky Hepburn...",Brad Davison,32,0,4,4,0,0,0,0,2,7,True,False,False,False,34.0,4277993,B. Davison,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,Badgers,https://a.espncdn.com/i/teamlogos/ncaa/500/275...,275,WISC,A00002,2022,2,2022-03-02,3,7,1,3,0,1,11.0,AWAY,2509,PUR,0.218750,13.61,0.387,70,19,Guard,67.4,31.760,31.133,70.000,70,78.0
2142,Wisconsin,401364469,"{'Johnny Davis', 'Tyler Wahl', 'Chucky Hepburn...",Ben Carlson,7,1,2,3,0,0,0,0,1,2,False,False,False,False,20.0,4433166,B. Carlson,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,Badgers,https://a.espncdn.com/i/teamlogos/ncaa/500/275...,275,WISC,A00002,2022,2,2022-03-02,1,2,0,1,0,0,11.0,AWAY,2509,PUR,0.285714,1.28,0.106,70,19,Big_man,10.6,10.311,7.686,7.333,70,78.0
2143,Wisconsin,401364469,"{'Johnny Davis', 'Tyler Wahl', 'Chucky Hepburn...",Chris Vogt,16,3,1,4,2,0,1,1,1,2,False,False,False,False,33.0,4279252,C. Vogt,https://a.espncdn.com/i/headshots/mens-college...,Center,C,Badgers,https://a.espncdn.com/i/teamlogos/ncaa/500/275...,2

In [15]:
# merged_team_opp_stats.to_csv("/drive/My Drive/Capstone Project College Basketball Data/ESPN_data/Features_and_boxscores/player_and_opp_stats.csv")
### File used for prediction model
# merged_team_opp_stats = pd.read_csv("/drive/My Drive/Capstone Project College Basketball Data/ESPN_data/Features_and_boxscores/player_and_opp_stats.csv",index_col=0)
### Drop duplicates
merged_team_opp_stats = merged_team_opp_stats.drop_duplicates()


In [16]:
merged_team_opp_stats['Home']=pd.get_dummies(merged_team_opp_stats.home_away, drop_first=True)
merged_team_opp_stats.loc[:, merged_team_opp_stats.columns!='home_away']
merged_team_opp_stats["Starter_numeric"] = pd.get_dummies(merged_team_opp_stats.starter, drop_first=True)
merged_team_opp_stats["Cleaned_position_bigs_guards"] = merged_team_opp_stats['athlete_position_abbreviation'].apply(convert_forwards_bigs)
merged_team_opp_stats
# merged_team_opp_stats[merged_team_opp_stats.athlete_position_abbreviation=='ATH'].team_short_display_name.unique()

,team_short_display_name,game_id,starting_lineup,athlete_display_name,min,oreb,dreb,reb,ast,stl,blk,to,pf,pts,starter,ejected,did_not_play,active,athlete_jersey,athlete_id,athlete_short_name,athlete_headshot_href,athlete_position_name,athlete_position_abbreviation,team_name,team_logo,team_id,team_abbreviation,team_color,season,season_type,game_date,FGM,FGA,3PM,3PA,FTM,FTA,largest_lead,home_away,opponent_id,opponent_abbrev,points_per_min,prev_player_ppg,prev_player_points_per_min,game_total_points,game_number,Cleaned_position_bigs_guards,cum_average_opp_points_to_role,cum_average_opp_min_allowed_to_role,prev_3_games_opp_min_allowed_to_role,prev_3_games__opp_points_allowed_to_role,total_pts_allowed,prev_total_opp_ppg,Home,Starter_numeric
0,Auburn,401082996,"{'Bryce Brown', 'Horace Spencer', 'Malik Dunba...",Chuma Okeke,36,1,4,5,1,4,1,2,2,22,True,False,False,False,5.0,4278052,C. Okeke,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,Tigers,https://a.espncdn.com/i/teamlogos/ncaa/500/2.png,2,AUB,03244d,2019,2,2019-03-09,9,14,3,5,1,1,6.0,AWAY,2633,TENN,0.611111,11.50,0.370,84,5,Big_man,63.5,32.650,32.733,61.333,84,74.5,0,1
1,Auburn,401082996,"{'Bryce Brown', 'Horace Spencer', 'Malik Dunba...",Horace Spencer,19,3,2,5,0,0,2,0,3,2,True,False,False,False,0.0,3908520,H. Spencer,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,Tigers,https://a.espncdn.com/i/teamlogos/ncaa/500/2.png,2,AUB,03244d,2019,2,2019-03-09,0,1,0,0,2,2,6.0,AWAY,2633,TENN,0.105263,3.00,0.276,84,5,Big_man,63.5,32.650,32.733,61.333,84,74.5,0,1
2,Auburn,401082996,"{'Bryce Brown', 'Horace Spencer', 'Malik Dunba...",Malik Dunbar,20,0,3,3,2,2,0,0,0,5,True,False,False,False,4.0,4278055,M. Dunbar,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,Tigers,https://a.espncdn.com/i/teamlogos/ncaa/500/2.png,2,AUB,03244d,2019,2,2019-03-09,1,3,1,2,2,4,6.0,AWAY,2633,TENN,0.250000,9.00,0.405,84,5,Guard,63.5,32.650,32.733,61.333,84,74.5,0,1
3,Auburn,401082996,"{'Bryce Brown', 'Horace Spencer', 'Malik Dunba...",Jared Harper,31,1,2,3,8,1,0,2,3,16,True,False,False,False,1.0,4066248,J. Harper,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,Tigers,https://a.espncdn.com/i/teamlogos/ncaa/500/2.png,2,AUB,03244d,2019,2,2019-03-09,5,13,2,7,4,4,6.0,AWAY,2633,TENN,0.516129,15.50,0.445,84,5,Guard,63.5,32.650,32.733,61.333,84,74.5,0,1
4,Auburn,401082996,"{'Bryce Brown', 'Horace Spencer', 'Malik Dunba...",Bryce Brown,33,0,3,3,2,2,0,1,1,16,True,False,False,False,2.0,3908521,B. Brown,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,Tigers,https://a.espncdn.com/i/teamlogos/ncaa/500/2.png,2,AUB,03244d,2019,2,2019-03-09,4,10,4,9,4,5,6.0,AWAY,2633,TENN,0.484848,11.75,0.367,84,5,Guard,63.5,32.650,32.733,61.333,84,74.5,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2141,Wisconsin,401364469,"{'Johnny Davis', 'Tyler Wahl', 'Chucky Hepburn...",Brad Davison,32,0,4,4,0,0,0,0,2,7,True,False,False,False,34.0,4277993,B. Davison,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,Badgers,https://a.espncdn.com/i/teamlogos/ncaa/500/275...,275,WISC,A00002,2022,2,2022-03-02,3,7,1,3,0,1,11.0,AWAY,2509,PUR,0.218750,13.61,0.387,70,19,Guard,67.4,31.760,31.133,70.000,70,78.0,0,1
2142,Wisconsin,401364469,"{'Johnny Davis', 'Tyler Wahl', 'Chucky Hepburn...",Ben Carlson,7,1,2,3,0,0,0,0,1,2,False,False,False,False,20.0,4433166,B. Carlson,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,Badgers,https://a.espncdn.com/i/teamlogos/ncaa/500/275...,275,WISC,A00002,2022,2,2022-03-02,1,2,0,1,0,0,11.0,AWAY,2509,PUR,0.285714,1.28,0.106,70,19,Big_man,10.6,10.311,7.686,7.333,70,78.0,0,0
2143,Wisconsin,401364469,"{'Johnny Davis', 'Tyler Wahl', 'Chucky Hepburn...",Chris Vogt,16,3,1,4,2,0,1,1,1,2,False,False,False,False,33.0,4279252,C. Vogt,https://a.espncdn.com/i/headshots/mens-college...,Center,C,Ba

In [17]:
total_points_allowed_to_position = merged_team_opp_stats.groupby(["game_id",'game_date',"team_short_display_name","team_id","opponent_abbrev","Starter_numeric","Cleaned_position_bigs_guards"]).agg(
          {'pts':'sum','min':'mean'}
      ).round(2).reset_index()
##### Remove teams that have ATH position players 
teams_with_ath_pos = (list(total_points_allowed_to_position[(total_points_allowed_to_position['Cleaned_position_bigs_guards']=="ATH")].team_short_display_name.unique()))
total_points_allowed_to_position = total_points_allowed_to_position[~total_points_allowed_to_position['team_short_display_name'].isin(teams_with_ath_pos)]
### Clean previous code for starters using .apply(lambda x: x.expanding().mean() for cumulative average and cleaner code - only need a few lines of code (instead of filtering and using cumsum())
### Runs faster too do the same thing for role(starter/reserve but do not have to include cleaned_position_bigs_guards - and merge with "merged_team_opp_stats" df instead of total_points_allowed_to_position.
cum_avg_pos =total_points_allowed_to_position.groupby(["opponent_abbrev",'Starter_numeric','Cleaned_position_bigs_guards'])['pts','min'].apply(lambda x: x.expanding().mean().round(3).shift())
cum_avg_pos = cum_avg_pos.rename(columns = {'pts':"prev_avg_pts_to_role_pos",'min':'prev_avg_min_to_role_position'})
merged_opp_stats_allowed= total_points_allowed_to_position.merge(cum_avg_pos,left_index=True, right_index = True)

# merged_opp_stats_allowed#### includes points and minutes to each position and starting/reserve role
cleaned_merged_opp_stats_allowed = merged_opp_stats_allowed.loc[:, ~merged_opp_stats_allowed.columns.isin(['pts','min'])]
cleaned_merged_opp_stats_allowed
box_scores_and_features = pd.merge(merged_team_opp_stats,cleaned_merged_opp_stats_allowed, on = ['game_id',	"game_date",	"team_short_display_name","team_id","opponent_abbrev","Starter_numeric","Cleaned_position_bigs_guards"]
                                   ,how = "left")
positions_encoding = pd.get_dummies(box_scores_and_features['Cleaned_position_bigs_guards'])
box_scores_and_features = box_scores_and_features.join(positions_encoding)
box_scores_and_features


C:\Users\Zach\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  if __name__ == '__main__':


,team_short_display_name,game_id,starting_lineup,athlete_display_name,min,oreb,dreb,reb,ast,stl,blk,to,pf,pts,starter,ejected,did_not_play,active,athlete_jersey,athlete_id,athlete_short_name,athlete_headshot_href,athlete_position_name,athlete_position_abbreviation,team_name,team_logo,team_id,team_abbreviation,team_color,season,season_type,game_date,FGM,FGA,3PM,3PA,FTM,FTA,largest_lead,home_away,opponent_id,opponent_abbrev,points_per_min,prev_player_ppg,prev_player_points_per_min,game_total_points,game_number,Cleaned_position_bigs_guards,cum_average_opp_points_to_role,cum_average_opp_min_allowed_to_role,prev_3_games_opp_min_allowed_to_role,prev_3_games__opp_points_allowed_to_role,total_pts_allowed,prev_total_opp_ppg,Home,Starter_numeric,prev_avg_pts_to_role_pos,prev_avg_min_to_role_position,Big_man,Guard
0,Auburn,401082996,"{'Bryce Brown', 'Horace Spencer', 'Malik Dunba...",Chuma Okeke,36,1,4,5,1,4,1,2,2,22,True,False,False,False,5.0,4278052,C. Okeke,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,Tigers,https://a.espncdn.com/i/teamlogos/ncaa/500/2.png,2,AUB,03244d,2019,2,2019-03-09,9,14,3,5,1,1,6.0,AWAY,2633,TENN,0.611111,11.50,0.370,84,5,Big_man,63.5,32.650,32.733,61.333,84,74.5,0,1,36.667,31.223,1,0
1,Auburn,401082996,"{'Bryce Brown', 'Horace Spencer', 'Malik Dunba...",Horace Spencer,19,3,2,5,0,0,2,0,3,2,True,False,False,False,0.0,3908520,H. Spencer,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,Tigers,https://a.espncdn.com/i/teamlogos/ncaa/500/2.png,2,AUB,03244d,2019,2,2019-03-09,0,1,0,0,2,2,6.0,AWAY,2633,TENN,0.105263,3.00,0.276,84,5,Big_man,63.5,32.650,32.733,61.333,84,74.5,0,1,36.667,31.223,1,0
2,Auburn,401082996,"{'Bryce Brown', 'Horace Spencer', 'Malik Dunba...",Malik Dunbar,20,0,3,3,2,2,0,0,0,5,True,False,False,False,4.0,4278055,M. Dunbar,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,Tigers,https://a.espncdn.com/i/teamlogos/ncaa/500/2.png,2,AUB,03244d,2019,2,2019-03-09,1,3,1,2,2,4,6.0,AWAY,2633,TENN,0.250000,9.00,0.405,84,5,Guard,63.5,32.650,32.733,61.333,84,74.5,0,1,24.667,33.723,0,1
3,Auburn,401082996,"{'Bryce Brown', 'Horace Spencer', 'Malik Dunba...",Jared Harper,31,1,2,3,8,1,0,2,3,16,True,False,False,False,1.0,4066248,J. Harper,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,Tigers,https://a.espncdn.com/i/teamlogos/ncaa/500/2.png,2,AUB,03244d,2019,2,2019-03-09,5,13,2,7,4,4,6.0,AWAY,2633,TENN,0.516129,15.50,0.445,84,5,Guard,63.5,32.650,32.733,61.333,84,74.5,0,1,24.667,33.723,0,1
4,Auburn,401082996,"{'Bryce Brown', 'Horace Spencer', 'Malik Dunba...",Bryce Brown,33,0,3,3,2,2,0,1,1,16,True,False,False,False,2.0,3908521,B. Brown,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,Tigers,https://a.espncdn.com/i/teamlogos/ncaa/500/2.png,2,AUB,03244d,2019,2,2019-03-09,4,10,4,9,4,5,6.0,AWAY,2633,TENN,0.484848,11.75,0.367,84,5,Guard,63.5,32.650,32.733,61.333,84,74.5,0,1,24.667,33.723,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2141,Wisconsin,401364469,"{'Johnny Davis', 'Tyler Wahl', 'Chucky Hepburn...",Brad Davison,32,0,4,4,0,0,0,0,2,7,True,False,False,False,34.0,4277993,B. Davison,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,Badgers,https://a.espncdn.com/i/teamlogos/ncaa/500/275...,275,WISC,A00002,2022,2,2022-03-02,3,7,1,3,0,1,11.0,AWAY,2509,PUR,0.218750,13.61,0.387,70,19,Guard,67.4,31.760,31.133,70.000,70,78.0,0,1,31.000,34.117,0,1
2142,Wisconsin,401364469,"{'Johnny Davis', 'Tyler Wahl', 'Chucky Hepburn...",Ben Carlson,7,1,2,3,0,0,0,0,1,2,False,False,False,False,20.0,4433166,B. Carlson,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,Badgers,https://a.espncdn.com/i/teamlogos/ncaa/500/275...,275,WISC,A00002,2022,2,2022-03-02,1,2,0,1,0,0,11.0,AWAY,2509,PUR,0.285714,1.28,0.106,70,19,Big_man,10.6,10.311,7.686,7.333,70,78.0,0,0,6.900,11.575,1,0
2143,

In [18]:
box_scores_and_features.to_csv("src\\Features_and_boxscores\\limited_sampled_player_and_opp_stats_with_position.csv")

Further analysis and additional features to consider- Get Scoring for team and opponent prior to each game/date. 